In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dataset-2/train.csv')
df.head()

,question,answer
0,What is the role of Siamese networks in domain...,Siamese networks can be used to align the repr...
1,What is a recurrent neural network (RNN)?,"An RNN handles sequence data, capturing tempor..."
2,What are Eigen values and Eigen vectors in the...,Eigen values represent the variance captured b...
3,What are Deep Q-Networks?,Deep Q-networks are neural networks that appro...
4,What properties of words can Word2Vec extract?,Word2Vec can extract and provide the most simi...


In [ ]:
df.shape

(1985, 2)

In [ ]:
print(df.question[0])
print(df.answer[0])

What is the role of Siamese networks in domain adaptation?
Siamese networks can be used to align the representations of source and target domains by minimizing the discrepancy between them, making the model more robust to domain shifts.


In [ ]:
df.isna().sum()

,0
question,0
answer,0


In [ ]:
import re
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers.optimization import AdamW
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import csv
from sklearn.model_selection import train_test_split

#import gradio as gr
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
import numpy as np

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
print(df_train.shape)
print(df_test.shape)

(1588, 2)
(397, 2)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "gpt2"

# Load the GPT tokenizer.
tokenizer = GPT2Tokenizer.from_pretrained(checkpoint, bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')

# default to left padding
tokenizer.padding_side = "left"

# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained(checkpoint, pad_token_id=tokenizer.eos_token_id)

tokenizer.pad_token=tokenizer.eos_token

NameError: name 'GPT2Tokenizer' is not defined

In [ ]:
from torch.nn import functional as F

max_seq_length = 512

tokenized_data = []
for _, row in df_train.iterrows():
    question = row['question']
    answer = row['answer']
    question = " ".join(question.split()[:300])
    out = "<Question>\n" + str(question) + " \nAnswer\n " + str(answer) + " \n<ANSWER_ENDED>\n "
    out_ids = tokenizer(out,truncation = True, max_length = max_seq_length)


    tokenized_data.append(out_ids)



In [ ]:
tokenized_data[0]

{'input_ids': [27, 24361, 29, 198, 2061, 318, 3895, 8705, 329, 20793, 1366, 30, 220, 198, 33706, 198, 27018, 8705, 329, 20793, 1366, 3407, 7605, 884, 355, 4441, 10375, 2846, 11, 13262, 803, 7869, 11, 393, 21004, 4253, 12409, 9633, 13, 220, 198, 27, 15037, 45532, 62, 49361, 29, 198, 220], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenized_data[1]

{'input_ids': [27, 24361, 29, 198, 2061, 318, 262, 9432, 2163, 287, 257, 14174, 1398, 7483, 30, 220, 198, 33706, 198, 383, 9432, 2163, 1690, 973, 318, 262, 2994, 2163, 11, 884, 355, 41968, 2994, 287, 20546, 10128, 393, 3272, 12, 298, 28338, 287, 2604, 2569, 20683, 11, 543, 5260, 262, 4049, 1022, 11001, 290, 4036, 14722, 13, 220, 198, 27, 15037, 45532, 62, 49361, 29, 198, 220], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
class CustomTextDataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        return self.tokenized_data[idx]

# Create a custom dataset
custom_dataset = CustomTextDataset(tokenized_data)

# Data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
type(data_collator)

transformers.data.data_collator.DataCollatorForLanguageModeling

In [ ]:
from transformers import EarlyStoppingCallback

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Capstone-Dataset2',
    learning_rate=0.01,
    lr_scheduler_type="linear",
    warmup_steps=100,
    #weight_decay=0.01,
    dataloader_num_workers=4,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    overwrite_output_dir=True,
    fp16=True,
    num_train_epochs=5,
    save_steps=1000,
    save_total_limit=2,
)

In [ ]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=custom_dataset,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Step,Training Loss


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


TrainOutput(global_step=250, training_loss=1.6846810302734374, metrics={'train_runtime': 100.3007, 'train_samples_per_second': 79.162, 'train_steps_per_second': 2.493, 'total_flos': 373159724544000.0, 'train_loss': 1.6846810302734374, 'epoch': 5.0})

In [ ]:
model_output_path = "/content/drive/MyDrive/Capstone-Dataset2/gpt-model"
 # Save the model
trainer.save_model(model_output_path)
# Save the tokenizer
tokenizer.save_pretrained(model_output_path)

('/content/drive/MyDrive/Capstone-Dataset2/gpt-model/tokenizer_config.json',
 '/content/drive/MyDrive/Capstone-Dataset2/gpt-model/special_tokens_map.json',
 '/content/drive/MyDrive/Capstone-Dataset2/gpt-model/vocab.json',
 '/content/drive/MyDrive/Capstone-Dataset2/gpt-model/merges.txt',
 '/content/drive/MyDrive/Capstone-Dataset2/gpt-model/added_tokens.json')

In [ ]:
def generate_answer(question):
    input_ids = tokenizer.encode(question, return_tensors="pt").to("cuda")

    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids).to("cuda")
    pad_token_id = tokenizer.eos_token_id

    #output = model[0].generate(
    output = model.generate(
        input_ids,
        max_new_tokens=100,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    start_index = decoded_output.find("Answer")
    end_index = decoded_output.find("<ANSWER_ENDED>")

    if end_index != -1:
        # Extract the text between "Answer" and "<ANSWER_ENDED>"
        answer_text = decoded_output[start_index + len("Answer"):end_index].strip()
        return answer_text
    else:
        # If "<ANSWER_ENDED>" is not found, return the text following "Answer"
        answer_text = decoded_output[start_index + len("Answer"):].strip()
        return answer_text

    #return tokenizer.decode(output[0], skip_special_tokens=True)
    #return tokenizer.decode(output, skip_special_tokens=True)


In [ ]:
model_output_path='/content/drive/MyDrive/Capstone-Dataset2/gpt-model'

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained(model_output_path)
tokenizer = GPT2Tokenizer.from_pretrained(model_output_path)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
model.to("cuda")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
question="What is deep learning?"
response = generate_answer(question)
print("Generated Answer:", response)

Generated Answer: Deep learning is a type of neural network architecture that uses neural networks to learn features from raw data.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
question="What is Machine Learning?"
response = generate_answer(question)
print("Generated Answer:", response)

Generated Answer: Machine Learning is a branch of Artificial Intelligence that involves computers and computers to learn and model.


In [ ]:
question="Explain overfitting in simple terms"
response = generate_answer(question)
print("Generated Answer:", response)

Generated Answer: Overfitting occurs when a model learns the noise in the training data, resulting in poor generalize to unseen data.


In [ ]:
question="Define K-Means clustering?"
response = generate_answer(question)
print("Generated Answer:", response)

Generated Answer: Clustering is a unsupervised learning algorithm that groups the data points into predefined classes, where each point is predefined.


In [ ]:
df_test=pd.read_csv("/content/drive/MyDrive/Dataset-2/test.csv")

In [ ]:
df_test

,question,answer
0,What is feature selection?,The process of selecting a subset of relevant ...
1,"What are Markov decision processes (MDP), and ...",MDPs are mathematical frameworks for modeling ...
2,What is batch normalization and how does it work?,Normalizes the layer's input over a mini-batch
3,How does Word2Vec create word embeddings?,Word2Vec creates word embeddings by training o...
4,What is the role of hidden layers in MLP?,Hidden layers in MLP help capture complex patt...
...,...,...
244,What is the Zipf Distribution in NLP?,The Zipf Distribution describes how a few elem...
245,What is transfer learning in deep learning?,Transfer learning adapts a pre-trained model t...
246,Explain the use of tokenization in NLP.,Tokenization is the process of breaking text i...
247,What is the Perceptron Learning Rule?,The Perceptron Learning Rule is an algorithm u...


In [ ]:
from tqdm import tqdm
tqdm.pandas(desc="Generating answers")
df_test['generated_answers'] = df_test['question'].progress_apply(lambda question: generate_answer(question))

Generating answers: 100%|██████████| 249/249 [03:47<00:00,  1.09it/s]


In [ ]:
df_test.head(5)

,question,answer,generated_answers
0,What is feature selection?,The process of selecting a subset of relevant ...,Feature selection is the process of selecting ...
1,"What are Markov decision processes (MDP), and ...",MDPs are mathematical frameworks for modeling ...,"MDP is a simple, and works by using the Q-func..."
2,What is batch normalization and how does it work?,Normalizes the layer's input over a mini-batch,Batch normalization standardizes the inputs by...
3,How does Word2Vec create word embeddings?,Word2Vec creates word embeddings by training o...,Word embeddings map words into dense vectors u...
4,What is the role of hidden layers in MLP?,Hidden layers in MLP help capture complex patt...,Hidden layers transform inputs into a higher-d...


In [ ]:
import re

def extract_answer(text):
    pattern = r'Answer(.*?)<ANSWER_ENDED>'
    result = re.search(pattern, text, re.DOTALL)

    if result:
        return result.group(1).strip()
    else:
        return text

In [ ]:
df_test['generated_answers'] = df_test['generated_answers'].apply(extract_answer)

In [ ]:
df_test.to_csv('model_Qtest_data_output.csv', index=False)

In [ ]:
!pip install -U accelerate
!pip install -U transformers
!pip install torch
!pip install evaluate
!pip install rouge
!pip install rouge_score
!pip install bert_score
!pip install gradio
!pip install typing_extensions

In [ ]:
from evaluate import load
import pyarrow.parquet as pq
from collections import defaultdict
from tqdm import tqdm
from bert_score import score

def metric(data):
    Question,Answer,Generated_Answer  = ["question","answer","generated_answers"]
    rouge = load("rouge")
    meteor = load("meteor")
    score_bert = defaultdict(int)
    bertscore = load("bertscore")

    for _,x in tqdm(data.iterrows(),total=len(data)):

        rouge_ans = rouge.compute(predictions=[x[Generated_Answer]],references=[x[Answer]])

        score_bert["rouge_1_ans"] += rouge_ans['rouge1']
        score_bert["rouge_2_ans"] += rouge_ans['rouge2']
        score_bert["rouge_L_ans"] += rouge_ans['rougeL']

        meteor_ans = meteor.compute(predictions=[x[Generated_Answer]],references=[x[Answer]])

        score_bert["meteor_ans"] += meteor_ans["meteor"]

        # BERT score
        results = bertscore.compute(predictions=[x[Generated_Answer]], references=[x[Answer]], lang="en")

        score_bert["bert_score_f1"] += results["f1"][0]

    print("The evalution scores are as follows:")

    for name,val in score_bert.items():
        print(f"{name}: {val/len(data)}")




In [ ]:
MBmetric(df_test)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
  0%|          | 0/249 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 249/249 [01:15<00:00,  3.31it/s]

The evalution scores are as follows:
rouge_1_ans: 0.26374662126122533
rouge_2_ans: 0.09114856916939472
rouge_L_ans: 0.227753382053244
meteor_ans: 0.2293496569399014
bert_score_f1: 0.8656602291218248


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [ ]:
model.push_to_hub("qa-gptmodel")

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Nishantc05/qa-gptmodel/commit/b99abb0833fa85cc0d18be5831228d21d954de44', commit_message='Upload model', commit_description='', oid='b99abb0833fa85cc0d18be5831228d21d954de44', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import GPT2Model, GPT2Config, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("Nishantc05/qa-gptmodel")
model.to("cuda")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
tokenizer.push_to_hub("qa-gptmodel")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Nishantc05/qa-gptmodel/commit/3cfc06762be54c79ba055621dbd1135e388d359c', commit_message='Upload tokenizer', commit_description='', oid='3cfc06762be54c79ba055621dbd1135e388d359c', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
print(device)

cuda


In [ ]:
import gradio as gr
from transformers import BartTokenizer, BartForConditionalGeneration


tokenizer = GPT2Tokenizer.from_pretrained("Nishantc05/qa-gptmodel", bos_token='<|startoftext|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
tokenizer.padding_side = "left"
tokenizer.pad_token=tokenizer.eos_token

model =  AutoModelForCausalLM.from_pretrained("Nishantc05/qa-gptmodel")
model.to(device)


def summarize_text(text):
    input_ids = tokenizer.encode(text, return_tensors="pt").to("cuda")

    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids).to("cuda")
    pad_token_id = tokenizer.eos_token_id

    #output = model[0].generate(
    output = model.generate(
        input_ids,
        max_new_tokens=100,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
    print(decoded_output)
    start_index = decoded_output.find("Answer")
    end_index = decoded_output.find("<ANSWER_ENDED>")

    if end_index != -1:
        # Extract the text between "Answer" and "<ANSWER_ENDED>"
        answer_text = decoded_output[start_index + len("Answer"):end_index].strip()
        return answer_text
    else:
        # If "<ANSWER_ENDED>" is not found, return the text following "Answer"
        answer_text = decoded_output[start_index + len("Answer"):].strip()
        return answer_text


# Create a Gradio interface
iface = gr.Interface(
    fn=summarize_text,
    inputs=gr.Textbox(lines=5, placeholder="Enter text here..."),
    outputs="text",
    title= "QA System with GPT2",
    description="Enter a question and get its answer using the GPT2 model."
)

# Launch the interface
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://38f02c03319f808fd8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
